# Scenario 1 - Which countries have the most Invoices?
```sql
SELECT BillingCountry, COUNT(*) AS Invoices 
FROM Invoice
GROUP BY BillingCountry
ORDER BY Invoices DESC
FETCH FIRST 5 ROWS ONLY;
```

In [2]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_1.sql'


BILLINGCOUNTRY                           |   INVOICES
---------------------------------------- | ----------
USA                                      |         91
Canada                                   |         56
France                                   |         35
Brazil                                   |         35
Germany                                  |         28



# Scenario 2 - Which city has the best customers?
```sql
SELECT BillingCity, SUM(Total)  AS InvoiceDollars 
FROM Invoice
GROUP BY BillingCity 
ORDER BY InvoiceDollars DESC
FETCH FIRST 3 ROWS ONLY;
```

In [3]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_2.sql'


BILLINGCITY                              | INVOICEDOLLARS
---------------------------------------- | --------------
Prague                                   |          90.24
Mountain View                            |          77.24
Paris                                    |          77.24



# Scenario 3 - Who is the best customer?
```sql
SELECT CustomerId, SUM(Total) AS Money_Spent 
FROM Invoice 
GROUP BY CustomerId 
ORDER BY Money_Spent DESC 
FETCH FIRST 1 ROWS ONLY;
```

In [4]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_3.sql'


CUSTOMERID | MONEY_SPENT
---------- | -----------
         6 |       49.62



# Scenario 4 - Find the first 5 customers by Id who are not in the US.
```sql
SELECT CustomerId, FirstName, LastName, Country
FROM Customer
WHERE not Country = 'USA'
ORDER BY CustomerId ASC
FETCH FIRST 5 ROWS ONLY;
```

In [5]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_4.sql'


CUSTOMERID | FIRSTNAME                                | LASTNAME             | COUNTRY
---------- | ---------------------------------------- | -------------------- | ----------------------------------------
         1 | Lu??s                                    | Gon??alves           | Brazil
         2 | Leonie                                   | K??hler              | Germany
         3 | Fran??ois                                | Tremblay             | Canada
         4 | Bj??rn                                   | Hansen               | Norway
         5 | Franti??ek                               | Wichterlov??         | Czech Republic



# Scenario 5 - Which employees are Sales Agents?
```sql
SELECT LastName, FirstName FROM Employee
WHERE Employee.Title = 'Sales Support Agent';
```

In [6]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_5.sql'


LASTNAME             | FIRSTNAME
-------------------- | --------------------
Peacock              | Jane
Park                 | Margaret
Johnson              | Steve



# Scenario 6 - What is the count of line items for Invoice ID 37?
```sql
SELECT COUNT(InvoiceLineId)
FROM InvoiceLine
WHERE InvoiceId = 37;
```

In [7]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_6.sql'


COUNT(INVOICELINEID)
--------------------
                   4



# Scenario 7 (Option 1 - Join) - Which artists have written the most Rock music?
```sql
SELECT Artist.ArtistId AS artistId, Artist.Name AS name, COUNT(Track.Name) AS Songs 
FROM Artist 
JOIN Album ON Album.ArtistId = Artist.ArtistId 
JOIN Track ON Album.AlbumId = Track.AlbumId 
JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock' 
GROUP BY Artist.ArtistId, Artist.Name, Genre.Name 
ORDER BY Songs DESC 
FETCH FIRST 10 ROWS ONLY; 
```

In [8]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_7_Option1.sql'


  ARTISTID | NAME                                               |      SONGS
---------- | -------------------------------------------------- | ----------
        22 | Led Zeppelin                                       |        114
       150 | U2                                                 |        112
        58 | Deep Purple                                        |         92
        90 | Iron Maiden                                        |         81
       118 | Pearl Jam                                          |         54
       152 | Van Halen                                          |         52
        51 | Queen                                              |         45
       142 | The Rolling Stones                                 |         41
        76 | Creedence Clearwater Revival                       |         40
        52 | Kiss                                               |         35

10 rows selected.



# Scenario 7 (Option 2 - Materialized View) - Which artists have written the most Rock music?
```bash
CREATE MATERIALIZED VIEW RockView
BUILD IMMEDIATE
AS
SELECT Artist.ArtistId AS artistId, Artist.Name AS name, COUNT(Track.Name) AS Songs 
FROM Artist
JOIN Album ON Album.ArtistId = Artist.ArtistId 
JOIN Track ON Album.AlbumId = Track.AlbumId 
JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock' 
GROUP BY Artist.ArtistId, Artist.Name, Genre.Name 
ORDER BY Songs DESC; 

ALTER TABLE CHINOOK.ROCKVIEW ADD SUPPLEMENTAL LOG DATA (ALL) COLUMNS;
ALTER MATERIALIZED VIEW RockView ADD CONSTRAINT PK_RockVIEW PRIMARY KEY (ArtistId);

SELECT * FROM RockView
FETCH FIRST 10 ROWS ONLY; 
```

In [9]:
! docker exec -i oracle /bin/sh -c 'echo exit | sqlplus -S chinook/Password1@localhost:1521/orclpdb1 @/opt/scenarios/scenario_7_Option2.sql'


  ARTISTID | NAME                                               |      SONGS
---------- | -------------------------------------------------- | ----------
        22 | Led Zeppelin                                       |        114
       150 | U2                                                 |        112
        58 | Deep Purple                                        |         92
        90 | Iron Maiden                                        |         81
       118 | Pearl Jam                                          |         54
       152 | Van Halen                                          |         52
        51 | Queen                                              |         45
       142 | The Rolling Stones                                 |         41
        76 | Creedence Clearwater Revival                       |         40
        52 | Kiss                                               |         35

10 rows selected.

